In [1]:
import uproot
import numpy as np

def load_root_file(file_path, branches=None, print_branches=False):
    all_branches = {}
    with uproot.open(file_path) as file:
        tree = file["tree"]
        # Load all ROOT branches into array if not specified
        if branches is None:
            branches = tree.keys()
        # Option to print the branch names
        if print_branches:
            print("Branches:", branches)
        # Each branch is added to the dictionary
        for branch in branches:
            all_branches[branch] = tree[branch].array(library="np")
    return all_branches

In [2]:
def process_branches(branches):
    # Concatenate all event arrays
    module_layers = np.concatenate(branches['module_layers'])
    module_subdets = np.concatenate(branches['module_subdets'])
    module_rings = np.concatenate(branches['module_rings'])
    module_eta = np.abs(np.concatenate(branches['module_eta']))

    # Initialize category numbers
    category_numbers = np.full_like(module_layers, -1)
    
    # Category masks
    mask1 = (module_layers <= 3) & (module_subdets == 5)
    mask2 = (module_layers >= 4) & (module_subdets == 5)
    mask3 = (module_layers <= 2) & (module_subdets == 4) & (module_rings >= 11)
    mask4 = (module_layers >= 3) & (module_subdets == 4) & (module_rings >= 8)
    mask5 = (module_layers <= 2) & (module_subdets == 4) & (module_rings <= 10)
    mask6 = (module_layers >= 3) & (module_subdets == 4) & (module_rings <= 7)

    category_numbers[mask1] = 0
    category_numbers[mask2] = 1
    category_numbers[mask3 | mask4] = 2
    category_numbers[mask5 | mask6] = 3

    # Eta classification
    eta_numbers = np.full_like(module_eta, -1)
    eta_numbers[module_eta < 0.75] = 0
    eta_numbers[(module_eta >= 0.75) & (module_eta < 1.5)] = 1
    eta_numbers[(module_eta >= 1.5) & (module_eta < 2.25)] = 2
    eta_numbers[(module_eta >= 2.25) & (module_eta < 3)] = 3

    # Split back into event-wise lists
    split_indices = np.cumsum([len(x) for x in branches['module_layers'][:-1]])
    category_numbers_split = np.split(category_numbers, split_indices)
    eta_numbers_split = np.split(eta_numbers, split_indices)

    # Add to branches dictionary
    branches['category_number'] = np.array(category_numbers_split, dtype=object)
    branches['eta_number'] = np.array(eta_numbers_split, dtype=object)
    
    return branches

In [3]:
import matplotlib
import matplotlib.pyplot as plt

font = {'size' : 20}

matplotlib.rc('font', **font)

def plot_histogram(data, title, xlabel, ylabel, occ_percentile=None):
    plt.figure(figsize=(10, 6))
    plt.hist(data, bins=50, edgecolor='black', alpha=0.7)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(True)
    plt.yscale('log')
    # Plotting a vertical line at the occupancy value
    if occ_percentile is not None:
        non_zero_data = data[data > 0]
        percentile_value = np.percentile(non_zero_data, occ_percentile)
        plt.axvline(percentile_value, color='red', linestyle='dashed', linewidth=1, label=f'{occ_percentile}th percentile: {percentile_value:.0f}')
        plt.legend()
    plt.show()

In [4]:
def compute_occupancies(branches, occupancy_variables, occ_percentiles, plot=False):
    for var, percentile in zip(occupancy_variables, occ_percentiles):
        matrix = [[0 for _ in range(4)] for _ in range(4)]
        for cat in range(4):
            for eta in range(4):
                # Extract relevant data
                data_to_plot = [
                    occupancy 
                    for sublist_cat, sublist_eta, sublist_occ in zip(
                        branches['category_number'], 
                        branches['eta_number'], 
                        branches[var]
                    )
                    for c, e, occupancy in zip(sublist_cat, sublist_eta, sublist_occ) 
                    if c == cat and e == eta
                ]
                data_to_plot = np.array(data_to_plot)
                non_zero_data = data_to_plot[data_to_plot > 0]
                
                if non_zero_data.size > 0:
                    percentile_value = np.percentile(non_zero_data, percentile)
                    
                    if plot:
                        # Plotting functionality remains
                        plot_histogram(data_to_plot, 
                                    f'{var} for Category {cat} and Eta {eta}', 
                                    'Occupancy', 
                                    'Frequency', 
                                    percentile)
                    else:
                        # Store values for matrix generation
                        matrix[cat][eta] = int(round(percentile_value))
        
        if not plot:
            # Generate matrix output
            matrix_name = var.replace('_occupancies', '_occupancy_matrix')
            print(f"      constexpr int {matrix_name}[4][4] = {{")
            for cat in range(4):
                row = matrix[cat]
                row_str = ", ".join(map(str, row))
                print(f"          {{{row_str}}},  // category {cat}")
            print("      };\n")

In [5]:
# Branches relevant to the occupancy selections
mod_occ_branches = ['module_layers', 'module_subdets', 'module_rings', 'module_eta',
                    'md_occupancies', 'sg_occupancies', 't3_occupancies', 't5_occupancies']

# Occupancy variables and their percentiles
occupancy_variables = ['md_occupancies', 'sg_occupancies', 't3_occupancies', 't5_occupancies']
percents = [99.99, 99.99, 99.9, 99.99]

# List of files to process
file_paths = [
    "500_new_occ_0p8.root",
    "500_new_occ_0p6.root"
]

for file_path in file_paths:
    print(f"\nProcessing file: {file_path}")
    
    # Load and process branches
    branches = load_root_file(file_path, mod_occ_branches)
    process_branches(branches)
    
    # Generate occupancy matrices
    compute_occupancies(branches, 
                       occupancy_variables, 
                       occ_percentiles=percents, 
                       plot=False)


Processing file: 500_new_occ_0p8.root
      constexpr int md_occupancy_matrix[4][4] = {
          {46, 42, 40, 37},  // category 0
          {132, 105, 0, 0},  // category 1
          {0, 17, 21, 0},  // category 2
          {0, 16, 20, 26},  // category 3
      };

      constexpr int sg_occupancy_matrix[4][4] = {
          {740, 314, 230, 60},  // category 0
          {1065, 693, 0, 0},  // category 1
          {0, 191, 201, 0},  // category 2
          {0, 67, 111, 75},  // category 3
      };

      constexpr int t3_occupancy_matrix[4][4] = {
          {668, 271, 105, 59},  // category 0
          {738, 310, 0, 0},  // category 1
          {0, 13, 5, 0},  // category 2
          {0, 92, 98, 42},  // category 3
      };

      constexpr int t5_occupancy_matrix[4][4] = {
          {161, 129, 135, 132},  // category 0
          {0, 0, 0, 0},  // category 1
          {0, 0, 0, 0},  // category 2
          {0, 0, 96, 112},  // category 3
      };


Processing file: 500_new_occ_0p6.root